# <font color="#114b98">      <div align="center">             Catégoriser automatiquement des questions

In [2]:
#python librairies
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics

### 1. Importation et préparation des données

In [3]:
#Import the cleaned dataset

path ="C:/Users/moumouni/Desktop/OC_P5_mlbis/Stack_questions_cleaned.csv"
data = pd.read_csv(path,sep=';')
for col in ['Title', 'Body', 'Tags', 'text_comb']:
     data[col] = data[col].apply(literal_eval) 
     
print(data.head(3))

# transform target tags
mlb = MultiLabelBinarizer()
mlb.fit(data.Tags)
y_mlb = mlb.transform(data.Tags)
print("Number of Tags:", len(mlb.classes_))
#
# data split
X_train, X_test, y_train, y_test = train_test_split(data.text_comb,\
                    y_mlb, test_size=0.25, random_state=42)


                            Title  \
0  [bracket, print, array, comma]   
1         [c, listing, directory]   
2  [return, datatables, wcf, net]   

                                                Body                     Tags  \
0  [slash, program, original, output, letter, por...   [java, array, android]   
1  [file, directory, cross, folder, c, scan, plat...                [file, c]   
2  [close, datatables, behaviorcontracts, datatab...  [web, service, c#, net]   

                                           text_comb  
0  [slash, program, original, output, letter, por...  
1  [cross, directory, file, folder, listing, c, s...  
2  [close, datatables, behaviorcontracts, datatab...  
Number of Tags: 50


### 2. Piple of the final modèle

In [4]:
# define model
# tfidf features
final_model = Pipeline([('tfidf', TfidfVectorizer(analyzer="word",
                             max_df=.98,
                             min_df= 2,
                             tokenizer=None,
                             preprocessor=' '.join,
                             stop_words=None,
                             lowercase=False)),
                        ('lr', OneVsRestClassifier(LogisticRegression()))])

#fit model
final_model.fit(X_train,y_train)

#prediction and score
y_score = final_model.predict_proba(X_test)
print('Score ROC_AUC:', metrics.roc_auc_score(y_test, y_score,multi_class="ovr"))

Score ROC_AUC: 0.966087207040356


In [5]:
from pickle import dump

#Modèle serialisation
dump(final_model, open("final_model.pkl","wb"))

In [6]:
# Example with one text
text = ['serialize','model','c#','class','data']
#threshold for labels with scores, fitted from the ROC 
threshold = 0.11 
y_pred = final_model.predict_proba(np.array([text,]))
tag = mlb.inverse_transform((y_pred>threshold)*1)

In [7]:
print('Tags proposés:', tag)

Tags proposés: [('c#', 'data', 'net', 'python')]
